In [35]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
def f(x):
  return 2*x**2 + 3*x - 4

In [37]:
f(3.0)

23.0

In [38]:
xs = np.arange(-10, 10, 0.5)
ys = f(xs)
plt.plot(xs, ys)

In [39]:
h = 0.0000001
x = -0.75
(f(x + h) - f(x))/h

2.042810365310288e-07

In [40]:
# complex'er case
a = 2.0
b = -3.0
c = 12.0
d = a*b + c
print(d)

6.0


In [41]:
h = 0.0001

# inputs
a = 2.0
b = -3.0
c = 12.0

d1 = a*b + c
c += h
d2 = a*b + c

print(f"d1: {d1}")
print(f"d2: {d2}")
print(f"slope: {(d2 - d1)/h}")

d1: 6.0
d2: 6.0001
slope: 0.9999999999976694


In [42]:
# Value object

class Value:
  def __init__(self, data, _children=(), _op='', label=""):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label


  def __repr__(self):
    return f"Value(data={self.data})"


  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other) # so constants can be added to Value instances as well
    sum = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * sum.grad
      other.grad += 1.0 * sum.grad
    sum._backward = _backward

    return sum


  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    prod = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * prod.grad
      other.grad += self.data * prod.grad

    prod._backward = _backward

    return prod


  # fallback function when doing something like const * Value
  # Python will check if Value can multiply const. if yes, it does rmul
  # instead of mul
  def __rmul__(self, other): # basically: other * self
    return self * other


  def __pow__(self, other):
    assert isinstance(other, (int, float))
    out = Value(self.data**other, (self, ), f'**{other}')

    def _backward():
      self.grad = other * self.data**(other-1) * out.grad
    out._backward = _backward

    return out


  def __truediv__(self, other):
    return self * other**-1


  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out


  def exp(self):
    x = self.data
    exp = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += exp.data * exp.grad # chain rule
    exp._backward = _backward

    return exp


  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)


  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [43]:
a = Value(2.0)
b = Value(4.0)
a - b

Value(data=-2.0)

In [44]:
# trying to visualize the graph

from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in the graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad, ), shape='record')
    if n._op:
      # if this node is a result of some operation, then create an operation node for it
      dot.node(name = uid + n._op, label = n._op)
      # connect this ndoe to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the operation node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [45]:
# backprop to a neuron

x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias of neuron
b = Value(6.88137, label='b')

# weighted sum
x1w1 = x1*w1; x1w1.label='x1*w1'
x2w2 = x2*w2; x2w2.label='x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label='n'
o = n.tanh(); o.label='o'
o.backward()

In [46]:
draw_dot(o)

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [30]:
# backprop to a neuron

x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias of neuron
b = Value(6.88137, label='b')

# weighted sum
x1w1 = x1*w1; x1w1.label='x1*w1'
x2w2 = x2*w2; x2w2.label='x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label='n'
# -----
e = (2*n).exp(); e.label='e'
o =  (e - 1) / (e + 1); o.label='o'
# -----
o.backward()
draw_dot(o)

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [31]:
o

Value(data=0.7071049876722272)

In [32]:
# Implementing this in PyTorch

import torch

x1 = torch.tensor([2.0]).double(); x1.requires_grad=True
w1 = torch.tensor([-3.0]).double(); w1.requires_grad=True
x2 = torch.tensor([0.0]).double(); x2.requires_grad=True
w2 = torch.tensor([1.0]).double(); w2.requires_grad=True
b = torch.tensor([6.881373587]).double(); b.requires_grad=True

n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.item())
o.backward()

print('----')
print(f"w1: {w1.grad.item()}")
print(f"x1: {x1.grad.item()}")
print(f"x2: {x2.grad.item()}")
print(f"w2: {w2.grad.item()}")

0.7071066904050358
----
w1: 1.0000002567688737
x1: -1.5000003851533106
x2: 0.5000001283844369
w2: 0.0
